### Data Exploration for Project Works

In [120]:
# !pip install requests
# !pip install sqlalchemy
# !pip install pandas

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3 as sqlite
import urllib.request
import sys
from zipfile import ZipFile
import requests
from io import BytesIO
import numpy as np
import os

#### Air Quality Data inspection

#### Load Data

In [3]:
# Load air_quality data
# URL of the file to download
data_url = "https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/who_ambient_air_quality_database_version_2024_(v6.1).xlsx?sfvrsn=c504c0cd_3&amp;download=true"
#file_path = "../../data/air_quality.xlsx" 

# Set up the request with a user-agent header
req = urllib.request.Request(
    data_url, 
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
)

# Make the request and save the file
with urllib.request.urlopen(req) as response, open(file_path, 'wb') as out_file:
    data = response.read()
    out_file.write(data)


# Read the Excel file
excel_data = pd.ExcelFile(file_path, engine='openpyxl')

# Display sheet names to verify
print("Available sheets:", excel_data.sheet_names)

sheet_name = 'Update 2024 (V6.1)'
usecols = ['iso3', 'country_name', 'city', 'year', 'version',
           'pm10_concentration', 'pm25_concentration', 'no2_concentration']
df_air = pd.read_excel(excel_data, sheet_name=sheet_name, engine='openpyxl', usecols= usecols)
df_air


FileNotFoundError: [Errno 2] No such file or directory: '../../data/air_quality.xlsx'

In [7]:
# URL of the file to download
data_url = "https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/who_ambient_air_quality_database_version_2024_(v6.1).xlsx?sfvrsn=c504c0cd_3&amp;download=true"
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
# Extract air_quality data
response = requests.get(data_url, headers=headers)
data = BytesIO(response.content)
# usecols = ['iso3', 'country_name', 'city', 'year', 'version',
#         'pm10_concentration', 'pm25_concentration', 'no2_concentration',
#         'type_of_stations', 'population']
df = pd.read_excel(data, engine="openpyxl",
                 sheet_name="Update 2024 (V6.1)")
df


,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,type_of_stations,population
0,ESP,Spain,A Coruna/ESP,2013.0,"V4.0 (2018), V4.0 (2018), V4.0 (2018), V4.0 (2...",23.238,11.491,28.841,"Urban, Urban, Suburban",246146.0
1,ESP,Spain,A Coruna/ESP,2014.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023)",27.476,15.878,19.575,"Urban, Urban, Suburban",247604.0
2,ESP,Spain,A Coruna/ESP,2015.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",25.515,14.004,22.731,"Urban, Urban, Suburban, Suburban",247604.0
3,ESP,Spain,A Coruna/ESP,2016.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",23.057,13.160,20.204,"Urban, Urban, Suburban, Suburban",247604.0
4,ESP,Spain,A Coruna/ESP,2017.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",26.849,14.114,21.543,"Urban, Urban, Suburban, Suburban",247604.0
...,...,...,...,...,...,...,...,...,...,...
40093,KOR,Republic of Korea,경기도/KOR,2017.0,V5.0 (2022),57.335,36.457,0.029,NaN,NaN
40094,KOR,Republic of Korea,경기도/KOR,2018.0,V5.0 (2022),50.838,31.586,0.027,NaN,NaN
40095,KOR,Republic of Korea,경기도/KOR,2019.0,V5.0 (2022),55.568,31.013,0.028,NaN,NaN
40096,CHN,China,虎英公园北/CHN,2018.0,V5.0 (2022),NaN,30.649,NaN,NaN,NaN


#### Transform Data

In [203]:
# Rename column to representatiove names
df_air = df_air.rename(columns={"iso3": "Code", "country_name": "Country", "city": "City", "year": "Year", "pm10_concentration": "PM10", "pm25_concentration": "PM25", "no2_concentration": "NO2"})
# Drop not in scope countries
selected_countries = ['Spain', 'Germany', 'Switzerland', 'Denmark', 'Norway', 'Belgium', 'Italy','France','Sweden','Netherlands (Kingdom of the)','Luxembourg']
df_air = df_air.loc[df_air["Country"].isin(selected_countries)]
df_air.loc[df_air["Country"] == "Netherlands (Kingdom of the)", "country_name"] = "Netherlands"
df_air["Year"] = df_air["Year"].astype(int)
df_air[["PM10", "PM25", "NO2"]].astype(float)
df_air[["PM10", "PM25", "NO2"]] = df_air[["PM10", "PM25", "NO2"]].round(2)
# Calculate Mean pollutant concentartions per country
df_air= df_air.groupby(["Code", "Country", "Year"]).agg({"PM10": "mean" , "PM25": "mean", "NO2":"mean"}).reset_index()
df_air = df_air.dropna()
df_air


,Code,Country,Year,PM10,PM25,NO2
0,BEL,Belgium,2010,26.07,16.66,24.94
1,BEL,Belgium,2013,22.76,14.53,22.44
2,BEL,Belgium,2014,20.53,13.58,20.48
3,BEL,Belgium,2015,20.50,12.61,19.97
4,BEL,Belgium,2016,20.24,12.74,20.28
...,...,...,...,...,...,...
107,SWE,Sweden,2017,13.57,4.63,11.21
108,SWE,Sweden,2018,16.02,5.61,11.65
109,SWE,Sweden,2019,15.40,5.51,10.98
110,SWE,Sweden,2020,14.53,5.45,8.58


### Temperature Data inspection

In [194]:
# Load temperature data

# URL of the ZIP file
zip_url = "https://fenixservices.fao.org/faostat/static/bulkdownloads/Environment_Temperature_change_E_All_Data.zip"

# Simulate a browser request to download the ZIP file
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(zip_url, headers=headers)

# Load the ZIP file content into a BytesIO object
zip_content = BytesIO(response.content)

with ZipFile(zip_content) as zip_file:
    # List all files in the ZIP file
    zip_file_list = zip_file.namelist()
    print("Files in ZIP:", zip_file_list)

    # Extract the first CSV file in the ZIP file
    first_file_name = zip_file_list[0]
    with zip_file.open(first_file_name) as first_file:
        df_temp = pd.read_csv(first_file, encoding='latin1')

df_temp

Files in ZIP: ['Environment_Temperature_change_E_All_Data.csv', 'Environment_Temperature_change_E_All_Data_NOFLAG.csv', 'Environment_Temperature_change_E_AreaCodes.csv', 'Environment_Temperature_change_E_Flags.csv']


,Area Code,Area Code (M49),Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1961F,...,Y2019,Y2019F,Y2020,Y2020F,Y2021,Y2021F,Y2022,Y2022F,Y2023,Y2023F
0,2,'004,Afghanistan,7001,January,7271,Temperature change,°c,0.745,E,...,2.911,E,-0.547,E,0.701,E,2.458,E,-2.200,E
1,2,'004,Afghanistan,7001,January,6078,Standard Deviation,°c,1.954,E,...,1.954,E,1.954,E,1.954,E,1.954,E,1.954,E
2,2,'004,Afghanistan,7002,February,7271,Temperature change,°c,-1.787,E,...,0.147,E,2.544,E,4.220,E,2.502,E,3.697,E
3,2,'004,Afghanistan,7002,February,6078,Standard Deviation,°c,2.591,E,...,2.591,E,2.591,E,2.591,E,2.591,E,2.591,E
4,2,'004,Afghanistan,7003,March,7271,Temperature change,°c,0.445,E,...,0.360,E,0.664,E,2.625,E,3.756,E,3.895,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9651,5873,'198,OECD,7018,JunJulAug,6078,Standard Deviation,°c,0.248,E,...,0.248,E,0.248,E,0.248,E,0.248,E,0.248,E
9652,5873,'198,OECD,7019,SepOctNov,7271,Temperature change,°c,0.039,E,...,1.214,E,1.376,E,1.763,E,1.289,E,2.254,E
9653,5873,'198,OECD,7019,SepOctNov,6078,Standard Deviation,°c,0.378,E,...,0.378,E,0.378,E,0.378,E,0.378,E,0.378,E
9654,5873,'198,OECD,7020,Meteorological year,7271,Temperature change,°c,0.169,E,...,1.291,E,1.431,E,1.589,E,1.237,E,1.706,E


#### Transform Data

In [195]:
# Rename columns to representative names
df_temp = df_temp.rename(columns={"Area": "Country", "Months": "Month"})
# Select only the required Element = Teperature change
df_temp = df_temp.loc[df_temp["Element"] == "Temperature change"]
# Get the list of duplicated year columns
drop_Y_columns = df_temp.filter(regex=r'^Y\d{4}F$').columns.tolist()
drop_columns = drop_Y_columns + ['Area Code', 'Area Code (M49)','Element Code', 'Element', 'Months Code']
# Drop unrequired columns
df_temp = df_temp.drop(drop_columns, axis=1)
selected_countries = ['Spain', 'Germany', 'Switzerland', 'Denmark', 'Norway', 'Belgium', 'Italy','France','Sweden','Netherlands (Kingdom of the)','Luxembourg']
df_temp = df_temp.loc[df_temp["Country"].isin(selected_countries)]
df_temp = df_temp.loc[df_temp["Month"].isin(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'Sepetember', 'October', 'November', 'December'])]
df_temp = pd.melt(df_temp, id_vars=["Country", "Month", "Unit"], var_name="Year" , value_name="Temperature Change" )
# Remove Y from values in column Year
df_temp["Year"] = df_temp["Year"].str[1:]
df_temp["Year"] = df_temp["Year"].astype(int)
# Select only required years (2013 - 2022()
df_temp = df_temp.loc[df_temp["Year"].between(2013,2022)]
# Calculate Mean temperature Change for year
df_temp= df_temp.groupby(["Country", "Year"]).agg({"Temperature Change": "mean"}).reset_index()
df_temp["Temperature Change"] = df_temp["Temperature Change"].round(2)
df_temp = df_temp.dropna()
df_temp

,Country,Year,Temperature Change
0,Belgium,2013,0.56
1,Belgium,2014,2.37
2,Belgium,2015,1.94
3,Belgium,2016,1.26
4,Belgium,2017,1.91
...,...,...,...
105,Switzerland,2018,2.49
106,Switzerland,2019,2.21
107,Switzerland,2020,2.44
108,Switzerland,2021,1.12


### Merge Datasets

In [204]:
df = pd.merge(df_air, df_temp, on= ["Country", "Year"], how= "inner")
df

,Code,Country,Year,PM10,PM25,NO2,Temperature Change
0,BEL,Belgium,2013,22.76,14.53,22.44,0.56
1,BEL,Belgium,2014,20.53,13.58,20.48,2.37
2,BEL,Belgium,2015,20.50,12.61,19.97,1.94
3,BEL,Belgium,2016,20.24,12.74,20.28,1.26
4,BEL,Belgium,2017,19.90,12.17,20.17,1.91
...,...,...,...,...,...,...,...
90,SWE,Sweden,2017,13.57,4.63,11.21,1.45
91,SWE,Sweden,2018,16.02,5.61,11.65,1.86
92,SWE,Sweden,2019,15.40,5.51,10.98,1.83
93,SWE,Sweden,2020,14.53,5.45,8.58,3.16


In [171]:
df_temp

,Country,Year,Temperature Change
0,Belgium,2013,0.555545
1,Belgium,2014,2.366909
2,Belgium,2015,1.943455
3,Belgium,2016,1.263091
4,Belgium,2017,1.914182
...,...,...,...
105,Switzerland,2018,2.487455
106,Switzerland,2019,2.210364
107,Switzerland,2020,2.438091
108,Switzerland,2021,1.115545


#### Load database

In [10]:
print(os.getcwd())
engine=create_engine('sqlite:///../../data/data.sqlite')
with engine.connect() as conn:
    df_temp = pd.read_sql('SELECT * FROM air_pollution', conn)
df_temp

c:\Users\Sahel Bloukat\Workspace\mein\DataEng\made\project


OperationalError: (sqlite3.OperationalError) no such table: air_pollution
[SQL: SELECT * FROM air_pollution]
(Background on this error at: https://sqlalche.me/e/20/e3q8)